In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [46]:
def tidy_split(df, column, sep=',', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value.strip())
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

def sale_activity(df, suffix=''):
    sale_act = tidy_split(df, 'Previous Sale Activity', sep='\n', keep=False)
    sale_act['Previous Sale Activity'] = sale_act['Previous Sale Activity'].str.replace('.', '')
    sale_activity_enddates = sale_act['Previous Sale Activity'].str[-11:]
    date_dict = {'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may':'05', 'jun': '06', 'jul': '07', 'ago': '08', 'sep':'09', 'oct': '10', 'nov': '11', 'dic': '12'}
    sale_activity_enddates = sale_activity_enddates.replace(date_dict, regex=True)
    sale_activity_enddates = sale_activity_enddates.replace('own-Unknown', np.nan).astype('datetime64')
    sale_act['end_dates'] = sale_activity_enddates
    sale_act['end_dates'].fillna(sale_act['Acq. Expires'], inplace=True)
    sale_act['client'] = sale_act['Previous Sale Activity'].str[:-25]
    sale_act = sale_act.pivot_table(index='Unique Id', values='end_dates', columns='client', aggfunc=max)
    return df.join(sale_act, on='Unique Id', how='left').sort_values(by='Unique Id'), [str(col) + suffix for col in sale_act.columns]

def cleanup(df, suffix=''):
    date_map = {'NOW': today, 'NOT AVAIL': pd.NaT, 'NOT ACQ': pd.NaT}
    df['Non-Exclusive Date'] = df['Non-Exclusive Date'].replace(date_map)
    df['Exclusive Date'] = df['Exclusive Date'].replace(date_map)
    df['Non-Exclusive Date'] = df.apply(lambda x: today if x['Available?'] == 'Avail NE' else x['Non-Exclusive Date'], axis=1)
    max_prev_sale_enddate = df['Previous Sale Activity'].str.extractall(r'(\d{2}\-\w{3}(\.)?\-\d{4})').replace('.', '')
    date_dict = {'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may':'05', 'jun': '06', 'jul': '07', 'ago': '08', 'sep':'09', 'oct': '10', 'nov': '11', 'dic': '12'}
    max_prev_sale_enddate = max_prev_sale_enddate.replace(date_dict, regex=True)
    max_prev_sale_enddate = max_prev_sale_enddate.astype('datetime64[ns]').reset_index().groupby('level_0')[0].max()
    max_prev_sale_enddate = max_prev_sale_enddate + pd.DateOffset(1)
    df['max_prev_sale_enddate'] = max_prev_sale_enddate
    max_prev_sale_enddate = df[['Exclusive Date', 'max_prev_sale_enddate']].max(axis=1)
    df['Exclusive Date'] = max_prev_sale_enddate
    df['Holdback'] = df['Holdback'].apply(lambda x: pd.Timestamp.date(x))
    mask = df['Holdback'] <= today
    df['Holdback'].loc[mask] = pd.NaT
    df['Non-Exclusive Date'] = df['Non-Exclusive Date'].apply(lambda x: pd.Timestamp.date(x))
    df['Exclusive Date'] = df['Exclusive Date'].apply(lambda x: pd.Timestamp.date(x))
    mask = (df['Non-Exclusive Date'] < df['Exclusive Date']) & (df['Non-Exclusive Date'] > today)
    df['Available?'].loc[mask] = df['Non-Exclusive Date'].loc[mask]
    df['Available?'] = df['Available?'].apply(lambda x: pd.Timestamp.date(pd.Timestamp(x)) if type(x) == int else x)
    df['First Run or Library'] = df['Is Reissue?'].fillna('First Run')
    df['First Run or Library'] = df['First Run or Library'].map({'Yes': 'Library', 'First Run': 'First Run'})

    df, sales = sale_activity(df, suffix)

    df.columns = list(df.columns[:13]) + [str(col) + suffix for col in df.columns[13:]]
    return df, sales

def process(df, sales, screeners, ratings):
    metadata = ['Title', 'Genre', 'Cast Member', 'Year Completed', 'Director',
                        'Project Type', 'Synopsis', 'Unique Id', 'Website', 'Original Format',
                        'Dialogue Language', 'Subtitle Language']

    metadata_df = df[metadata].drop_duplicates().copy()

    agg_dict = {'Region': lambda x: ' & '.join(x)}

    for col in df.columns:
        for colname in ['First Run or Library', 'Available', 'Holdback', 'Note', 'Acq. Expires', 'Previous Sale Activity']:
            if colname in col:
                agg_dict[col] = 'first'

    sales = [sale for sale in sales if not sale.startswith('_')]

    for sale in sales:
        agg_dict[sale] = 'first'

    for col in df.columns:
        if 'Exclusive Date' in col:
            df[col].fillna(pd.Timestamp.max, inplace=True)

    df = df.groupby(['Unique Id']+[col for col in df.columns if 'Date' in col]).agg(agg_dict).reset_index()

    region_mapping = {'Brazil & Latin America excluding Brazil & Mexico & Mexico': 'All Latam',
            'Brazil & Mexico & Latin America excluding Brazil & Mexico': 'All Latam',
            'Mexico & Brazil & Latin America excluding Brazil & Mexico': 'All Latam',
            'Mexico & Latin America excluding Brazil & Mexico & Brazil': 'All Latam',
            'Latin America excluding Brazil & Mexico & Mexico & Brazil': 'All Latam',
            'Latin America excluding Brazil & Mexico & Brazil & Mexico': 'All Latam',
            'Latin America excluding Brazil & Mexico & Mexico': 'Latin America excluding Brazil',
            'Mexico & Latin America excluding Brazil & Mexico': 'Latin America excluding Brazil',
            'Latin America excluding Brazil & Mexico & Brazil': 'Latin America excluding Mexico',
            'Brazil & Latin America excluding Brazil & Mexico': 'Latin America excluding Mexico',
             'Brazil': 'Brazil',
             'Mexico': 'Mexico',
             'Mexico & Brazil': 'Mexico & Brazil',
             'Brazil & Mexico': 'Mexico & Brazil',
             'Latin America excluding Brazil & Mexico': 'Latin America excluding Brazil & Mexico',}

    df['Region'] = df['Region'].map(region_mapping)
    df = pd.merge(df, metadata_df, on='Unique Id')
    df = pd.merge(df, screeners, on='Unique Id', how='left')
    df = pd.merge(df, ratings, on='Unique Id', how='left')
    acq_exp_cols = [col for col in df.columns if 'Acq. Expires' in col]
    expiry_dates = df[acq_exp_cols[0]].apply(lambda x: pd.Timestamp.date(x))
    df.drop(acq_exp_cols, axis=1, inplace=True)
    df['Acq. Expires'] = expiry_dates
    date_cols = [col for col in df.columns if 'Date' in col]
    df[date_cols] = df[date_cols].replace({pd.Timestamp.max: pd.NaT})

    for col in date_cols:
        mask = df[col] >= df['Acq. Expires']
        df[col].loc[mask] = pd.NaT
        df[col] = df[col].apply(lambda x: 'Now' if x == today else x)
        
    df['Year'] = df['Year Completed']
    df[sales] = df[sales].apply(lambda x: x.apply(lambda x: str(x)[:10] if str(x) != 'nan' else ''))
    df[[col for col in df.columns if 'Available' in col]] = df[[col for col in df.columns if 'Available' in col]].apply(lambda x: x.apply(lambda x: pd.Timestamp.date(pd.Timestamp(x)) if type(x) == int else x))
    df[[col for col in df.columns if 'Available' in col]] = df[[col for col in df.columns if 'Available' in col]].apply(lambda x: x.apply(lambda x: str(x).replace(' 00:00:00', '')))
    df[[col for col in df.columns if 'Holdback' in col]] = df[[col for col in df.columns if 'Holdback' in col]].apply(lambda x: x.apply(lambda x: str(x).replace(' 00:00:00', '')))
    df[[col for col in df.columns if 'Date' in col]] = df[[col for col in df.columns if 'Date' in col]].apply(lambda x: x.apply(lambda x: str(x).replace(' 00:00:00', '')))
    df = df.apply(lambda x: x.apply(lambda x: '' if (str(x) == 'NaT' or str(x) == 'None' or str(x) == '0001-01-01') else x))

    return df

In [47]:
svod_avails = pd.read_excel('/Users/alejandroleda/Downloads/Availability by Region with Reissues - SVOD.xlsx', skiprows=1)

ptv_avails = pd.read_excel('/Users/alejandroleda/Downloads/Availability by Region with Reissues - Basic Pay TV.xlsx', skiprows=1)

ptv_local_avails = pd.read_excel('/Users/alejandroleda/Downloads/Availability by Region with Reissues - SVOD.xlsx', skiprows=1)

screeners = pd.read_excel('/Users/alejandroleda/Documents/Ledafilms/Misc/Temp/Project Data ID.xlsx')
screeners.dropna(axis=0, subset=['Unique Identifier'], inplace=True)
screeners['Unique Id'] = screeners['Unique Identifier'].astype(int)
screeners.drop(['Unique Identifier', 'Title', 'Web Site'], axis = 1, inplace=True)

ratings = pd.read_excel('/Users/alejandroleda/Documents/Ledafilms/Misc/Temp/Ratings & Titles.xls')
ratings.dropna(axis=0, subset=['Unique Identifier'], inplace=True)
ratings['Unique Id'] = ratings['Unique Identifier'].astype(int)
ratings.drop(['Unique Identifier', 'Title', 'Imdb'], axis = 1, inplace=True)

today = pd.Timestamp.date(pd.Timestamp.today())

svod_avails, svod_sales = cleanup(svod_avails, '_SVOD')
ptv_avails, ptv_sales = cleanup(ptv_avails, '_PanRegionalPayTV')
ptv_local_avails, ptv_local_sales = cleanup(ptv_local_avails, '_LocalPayTV')

ptv_avails = ptv_avails.merge(ptv_local_avails, on=list(ptv_avails.columns[:13]), how='left')
merged_df = ptv_avails.merge(svod_avails, on=list(ptv_avails.columns[:13]), how='left')

sales = svod_sales+ptv_sales+ptv_local_sales
sales = [sale for sale in sales if not sale.startswith('_')]

df = process(merged_df, sales, screeners, ratings)

cols_ordered = ['Project Type', 'Unique Id', 'Title', 'Region', 'Year', 'Genre', 'Cast Member',
                            'Director',  'Synopsis', 'Website',
                            'Original Format', 'Dialogue Language', 'Subtitle Language',

                            'First Run or Library_PanRegionalPayTV',
                            'Available?_PanRegionalPayTV', 'Non-Exclusive Date_PanRegionalPayTV',
                            'Exclusive Date_PanRegionalPayTV', 'Note_PanRegionalPayTV',
                            'Holdback_PanRegionalPayTV',

                            'First Run or Library_LocalPayTV',
                            'Available?_LocalPayTV', 'Non-Exclusive Date_LocalPayTV',
                            'Exclusive Date_LocalPayTV', 'Note_LocalPayTV',
                            'Holdback_LocalPayTV',

                            'First Run or Library_SVOD',
                            'Available?_SVOD', 'Non-Exclusive Date_SVOD',
                            'Exclusive Date_SVOD', 'Note_SVOD',
                            'Holdback_SVOD',

                            'Acq. Expires',
                            'Link','Password', 'IMDB Link', 'US Box Office', 'LATAM Box Office', 'USA ',
                            'Mexico', 'Brazil', ' Argentina', 'Bolivia', 'Chile', 'Colombia ',
                            'Costa Rica', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras',
                            'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Dominican Republic',
                            'Uruguay', 'Venezuela']

/Users/alejandroleda/opt/anaconda3/envs/generalCondaEnv/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-46-82f8b6944bc7>:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  sale_act['Previous Sale Activity'] = sale_act['Previous Sale Activity'].str.replace('.', '')
/Users/alejandroleda/opt/anaconda3/envs/generalCondaEnv/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [48]:
df[cols_ordered + sales].to_excel('/Users/alejandroleda/Documents/Ledafilms/Misc/Temp/SVOD-PayTV avails.xlsx')